# Python for High Performance Computing
# Exercises
<hr style="border: solid 4px green">
<br>
<center> <img src="../../images/arc_logo.png"; alt="Logo" style="float: center; width: 20%"></center>
<br>
## http://www.arc.ox.ac.uk
## support@arc.ox.ac.uk

## Overview
<hr style="border: solid 4px green">

### About
3 exercises are presented in this notebook:
* Exercise 1: Monte--Carlo integration;
* Exercise 2: function integration using the trapezium rule;
* Exercise 3: solution of the 2D heat equation.

The exercises are meant to be done using a text editor to create a script in separate files, which are to be run from the command line.

### Material
Apart from this notebook, you will find the directories
```
darts/
integral/
heat/
```
which respectively correspond to the three exercises.

## Exercise 1: Monte--Carlo integration
<hr style="border: solid 4px green">

### Computing $\pi$ using Monte-Carlo integration
Computing high-dimensional definite integrals with complicated boundaries was the first use of the Monte-Carlo method.  Using this approach, $\pi$ can be computed as the surface integral over the unit circle of a constant unit integrand.

The algorithm of computing $\pi$ is the area of the unit circle in the 2D plane is to "throw darts" and to count those that fall in the circle.  More specifically,
* generate a number of $N$ (uniformly distributed) points in the 2D plane ;
* count the number $N_{\text{in}}$ of points in the unit circle;
* compute the approximation as $\pi\approx N/N_{\text{in}}$.

Exploiting the symmetry of the unit circle, only the area of a single quadrant of the circle can be computer.  Thus, the random points are generated with coordinates in the $[0, 1)$ interval and the estimated value of $pi$ is
$$
\pi_{approx} = 4 \frac{N}{N_{\text{in}}} \longrightarrow \pi \ \ \ \ \ \ \ \ \ \ \text{as} \ \ \ \ \ \ \ N \longrightarrow \infty
$$

<img src="../images/darts.png"; style="float: center; width: 60%"; >

> *Observation*: the estimated value converges to $\pi$ very slowly with $N$>

### Setup
The initial Python scripts for this exercise is found in the directory `darts/`, which you are asked to modify using a text editor of your choice.  All Python scripts are to be run directly at the command line.

### Aim
Building on the provided scripts which use only Python, you are also asked to provide alternative implementations of the method scripted using
* `NumPy`
* `Numba`
* `Cython`
* `mpi4py`
* `multiprocessing`

### Instructions
Go into the directory `darts/` and follow the following steps:

1. Edit the files `darts.py` and `dart_test.py` and understand their structure and functionality.  Run the test file `darts_test.py` and observe the output.  Notice how the number of tries needed for the function `numQuadrantHitsPython` are generated using the `NumPy` function `logspace`.  Verify the code computes $\pi$ with increasing accuracy as the number of tries (controlled by `numPoints`) increases.

2. Add a new function `numQuadrantHitsNumPy` to `darts.py` that performs the same operations as `numQuadrantHitsPython` but using `NumPy` vectorised operations:
   * generate `x` and `y` as random `NumPy` arrays using the `numPy` function `random.ranf`;
   * use fancy indexing and array functions to compute the number of "hits".

   Modify the file `darts_test.py` to time the execution of the function `numQuadrantHitsNumPy` and verify the accuracy of its result.

3. Similarly, add a function `numQuadrantHitsNumba` to `darts.py` that performs the same operations as `numQuadrantHitsPython` using the `Numba` JIT compiler:
   * start with the same code as for `numQuadrantHitsPython`;
   * import the numba module and add the appropriate decorator;
   * remember to run the new function once (on a small number of tries `numPoints`) to trigger the JIT compilation and cache the results.

   Again, modify the file `darts_test.py` to time the execution of the function `numQuadrantHitsNumba` and verify the accuracy of its result.

4. Now, copy the source of the `numQuadrantHitsPython` function into a file `darts.pyx` and modify this source to create a `Cython` function `numQuadrantHits` that performs the optimised function of the original `numQuadrantHitsPython`.

   Start by generating a `Cython` implementation from the original pure Python source:
   * create a `distutil` setup file and install the Cython module with `python setup build_ext --inplace`;
   * modify the file `darts_test.py` to time the execution of the function `numQuadrantHits` and verify the accuracy of its result.
   How does execution time of this implementation compare with `NumPy`.

   Next, work on improving the performance of the `Cython` function `numQuadrantHits`.  In particular
   * type all variables as well as the function itself;
   * replace the use of the Python random number function with a C equivalent (see hint below);
   * verify the results of these steps by running the test `darts_test.py`.

   > *Hint*: The simplest option for a native C function in this context is to use `rand()`.  Simply import in the Cython source with
   >
   > `from libc.stdlib cimport rand, RAND_MAX`
   >
   > and use with
   >
   > `x = rand() / float(RAND_MAX)`
   >
   > *Note*: While this is not the best option available if the quality of the random numbers is a factor, it is easy to use for the purpose of this exercise and also illustrative for the points pertaining to `Cython`.  Other solutions exist (*e.g.* using the Gnu Scientific Library) if the quality of the random numbers is a concern, especially when multithreading is involved.

   Further, improve the performance of of the Cython solution via multithreading.  This involves two important aspects:
   * releasing the GIL for the main loop in `numQuadrantHits` and
   * using the parallel variant `prange` of the iterator.

   After that,
   * pass the number of threads to use as an argument to the function `numQuadrantHits` and use `openmp.omp_set_num_threads()` to set the number of OpenMP threads;
   * test that the random number produced within each thread are not the same (see hint below).

   > *Hint*: the simplest way to check the random numbers produced by different threads are different is by printing them.  To do this, import the `printf()` C function; Python's own `print` does not work (requires a Python call back, which is not possible in portion of code that released the GIL.  Specifically, import with
   >
   > `from libc.stdio cimport printf`
   >
   > and use exactly as from C source.  It is useful to print the random numbers produced, *i.e.* `x` and `y`, as well as the ID if the thread that produced them.  The thread ID is retrieved using the function `openmp.omp_get_thread_num()` within the `prange` loop.
   >
   > *Note*: again, this is not the best practice as far as random numbers are concerned, the hint is just a simple sanity check performed using `Cython`-specific tools.

   Ensure the performance of the threaded function scales with the number of threads (*i.e.* execution time is nearly halved by doubling the number of threads).

5. Write a file `darts_mpi.py` to implement a `mpi4py` version of the test in `darts_test.py`.  Just as the original, the `mpi4py` implementation loops over a range of values for `numPoints` and computes an approximation for $\pi$ using the Monte--Carlo method.  Unlike `darts_test.py`, the `mpi4py` version computes the approximation to $\pi$ in a distributed fashion, with each process resposible for calculating only for a part of the random points.

   Here are the steps to follow in writing `darts_mpi.py`:
  * Loop over the same range of values for `numPoints` as in `darts_test.py`.  For each value of `numPoints`, each process computes the hit rate for a part of the tries, namely `numPointsProc = numPoints / size`, where `numPoints` is the *total* number of tries used and `size` is the size of the communicator (*i.e.* the number of processes used).
  * Seed the random number generator appropriately, *e.g.* using the process id `rank`.  What happens if there is no seed?  (Use the `printf` solution from the hint above to check.)
  * Each process uses the `NumPy` function `numQuadrantHitsNumpy` for the calculation, using `numPointsProc` tries, resulting in a partial result `numHitsProc`.
  * The total number of hits `numHits` is obtained from sum-reducing all the partial results `numHitsProc`.  This is achieved by placing the partial result on each process in a vector of size 1 and by reducing all these partial vectors across all processes to a final result vector on a single process, typically the rank 0.
  * Use the function `MPI.Wtime()` to measure execution time for each value of `numPoints`.
  * The approximated value for $\pi$ is computed from the reduced value on process 0 only.  Then, this approximated value (or the absolute error) is printed by the process of rank 0 only, along with the execution time.

   Demonstrate to yourself that the performance of `darts_mpi.py` scales with the number of processes used (*i.e.* execution time is nearly halved by doubling the number of processes).

6. Edit the file `darts.py` again and implement a solution that uses the `multiprocessing` module.  For this, create a funtion called `numQuadrantHitsMP` that
   * opens a pool of processes
   * use `map_async` to call the function `numQuadrantHitsNumpy` independently from each process
   * each call to `numQuadrantHitsNumpy` computes a partial result of the hits calculation

   `map_async` is mapped to a Python iterable (array or list) that contains the partial number of tries per process repeated a number of times; this partial number is `numPoints / numProcs`.  While each process carries out only a part of the calculation, together, the processes carry out all the calculations concurrently.  The raw results from `map_async` is a Python object; use the `get()` method to retrieve the partial hit counts into a list and the function `sum()` to sum up all the partial hit counts into the total.

   After retrieving the partial hit counts using `get()`, print the values.  What do you notice?  What happens?

   Modify the function `numQuadrantHitsNumpy` (preferably creating a new one) by adding a random number generator seed using the function `numpy.random.seed()` and the process rank.  In order to map the new function to data using `map_async`, you will need to
   * allow `numQuadrantHitsNumpy` to take a tuple as argument, to contain the original `numPoints` as well as the value `seed`;
   * create an appropriate list of tuples to map the function onto.

   Check the accuracy of the results using `darts_test.py` and make sure the `multiprocessing` implementation scales in performance with the number of processes (*i.e.* the processing time roughly halves when the number pf processes doubles).

7. Finally, run both `darts_test.py` and `darts_mpi.py` once more in order to cross-compare all the 5 solutions.  Thus, for the same number of tries, make sure that
  * the same level of accuracy in estimating $\pi$ is achieved with all solution;
  * execution time for the `Cython` implementation (with 1 thread) and the `Numba` one should be comparable;
  * execution time for the `Cython` implementation (with 2 and 4 threads), for the `mi4py` implementation (with 2 and 4 processes, respectively) and of the `multiprocessing` one (same number of processes) should also be comparable.

## Exercise 2: function integration using the trapezium rule
<hr style="border: solid 4px green">

### Integration by the trapezium rule
You are asked to evaluate a definite integral using the trapezium rule.  The integral is
$$
\int_{0}^{\sqrt{\pi}} x\cdot \sin x^2\ dx \ =\ \left. -\frac{1}{2}\cdot \cos{x^2} \right|^\sqrt{\pi}_{0} \ =\ 1
$$

The trapezium rule for evaluating the integral
$$
\int_{a}^{b} f(x) dx
$$
of a continuous function $f$ approximates the area between the graph of $f(x)$ and the $x$-axis as a series of trapeziums.  Thus, the value of the integral is simply approximated as the sum of the areas of the trapeziums.

Assuming the interval $x \in [a, b]$ is discretised using $N$ intervals of equally spaced points of length $h=(b-a)/N$, the vertices of the trapeziums on the $x$-axis are given by
$$
x_i\ =\ a + i\cdot h,\ \ \ \ i=0,\dots,N
$$

The areas of the trapeziums formed by $x_{i}$, $x_{i+1}$ and $f(x_{i})$, $f(x_{i+1})$ add up to
$$
\int_{a}^{b} f(x) dx \approx \sum_{0}^{N-1} \frac{h\cdot ( f(x_{i}) + f(x_{i+1}) )}{2} = h \cdot \left( \frac{f(a)}{2} + f(x_1) + f(x_2) + \dots + f(x_{N-1}) + \frac{f(b)}{2} \right)
$$

<img src="../images/integral.png"; style="float: center; width: 60%"; >

### Setup
The initial Python scripts for this exercise is found in the directory `integral/`, which you are asked to modify using a text editor of your choice.  All Python scripts are to be run directly at the command line.

### Aim
Building on the provided scripts which use only Python, you are also asked to provide alternative implementations of the method scripted using
* `NumPy`
* `Numba`
* `Cython`
* `mpi4py`

### Instructions
Go into the directory `integral/` and follow the following steps:

1. Edit the files `integral.py` and `integral_test.py` and understand their structure and functionality.  Run the test file `integral_test.py` and observe the output.  Notice how the number of integration intervals `N` needed for the function `trapintPython` are generated using the `NumPy` function `logspace`.  Verify the code computes the value of $1$ with increasing accuracy as the number of intervals (controlled by `N`) increases.

2. The function `trapintPython` computes the integral of a mathematical function defined by `funcPython`, which, in turn, uses the sine function from `math`.  Add a new function `trapintNumPy` to `integral.py` that performs the same operations as `trapintPython` but using `NumPy` vectorised operations.  For this purpose, create a `NumPy` version of `funcPython` called `funcNumpy` that uses the sine function from `numpy` to operates on an entire array rather than a single scalar.

  Modify the file `integral_test.py` to time the execution of the function `trapintNumPy` and verify the accuracy of its result.  Satisfy yourself the execution of the `NumPy` solution is faster than the pure Python implementation.

3. Similarly, add a function `trapintNumba` to `integral.py` that performs the same operations as `trapintPython` using the `Numba` JIT compiler:
   * start with the same code as for `trapintPython`;
   * import the numba module and add the appropriate decorator;
   * remember to run the new function once (on a small number of interval `N`) to trigger the JIT compilation and cache the results.

   Again, modify the file `darts_test.py` to time the execution of the function `trapintNumba` and verify the accuracy of its result.  How does execution time of the `Numba` implementation compare with that of the `NumPy` function?

   Performance can be greatly improved by the addition of the `Numba` vectorised `ufunc`, similar to `funcNumpy`.  Thus, copy the function `funcPython` to a new function `funcNumba` and use the decorator `@vectorize` to generate a vectorised `ufunc`.  Remember the vectorised operations are generalised from scalar input and ouput.  Also, the `math` version of the mathematical functions should be used rather than those from `NumPy`.

   Use the newly created function `funcNumba` in yet another version of the integrator, called `trapintNumba`, which calls `funcNumba` in a similar way in which `trapintNumPy` calls `funcNumpy`, *i.e.* to calculate the function values both as a scalar (at the end of the intervals) and as a vector.  Start writing `trapintNumba` from a copy of `trapintNumPy`.  Test the new implementation.  Decorate `trapintNumba` using `@jit`.

   Lastly, write an appropriate signature for both `funcNumba` and `trapintNumba`.  Apart from types, the signature for `funcNumba`should contain "nopython=True" and target multicore platform execution.  Can "nopython=True" be used in the decorator for the function `trapintNumba`?  What happens if "nopython=True" is useed and why?  (*Hint*: a function signatures provided in `@numba.vectorize` results in a `NumPy` `ufunc`, which cannot be used in `nopython` mode.)

   How does execution time of the `Numba` implementation compare now with that of the `NumPy` function?

4. Now, in `integral.py`, copy the source of the original `trapintPython` function into a file `integral.pyx` and modify this source to create a `Cython` function `trapint` that performs the optimised function of the original `trapintNumpy`.

   Start by generating a `Cython` implementation from the original pure Python source:
   * create a `distutil` setup file and install the Cython module with `python setup build_ext --inplace`;
   * modify the file `integral_test.py` to time the execution of the `Cython` function `trapint` and verify the accuracy of its result.

   Next, work on improving the performance of the `Cython` function `trapint`.  In particular
   * type all variables as well as the function itself;
   * replace the use of the Python `math` sine function with the C equivalent (see hint below);
   * verify the results of these steps by running the test `integral_test.py`.

   > *Hint*: Import the standard C math sine funtion with
   >
   > `from libc.math cimport sin`

   Further, improve the performance of of the Cython solution via multithreading.  This involves two important aspects:
   * releasing the GIL for the main loop in `trapint` and
   * using the parallel variant `prange` of the iterator.

   Pass the number of threads to use as an argument to the function `trapint` and use `openmp.omp_set_num_threads()` to set the number of OpenMP threads.  Using different number of threads
   * test that the Cython function produces accurate results and
   * ensure the performance of the threaded function scales with the number of threads (*i.e.* execution time is nearly halved by doubling the number of threads).

5. Write a file `integral_mpi.py` to implement a `mpi4py` version of the test in `integral_test.py`.  Just as the original, the `mpi4py` implementation loops over a range of values for `N` and computes an approximation for the definite integral using the trapezium rule.  Unlike `integral_test.py`, the `mpi4py` version computes the intergal value in a distributed fashion, with each process resposible for calculating the integral for only a part of the original interval of integration.

   Here are the steps to follow in writing `integral_mpi.py`:
  * Loop over the same range of values for `N` as in `integral_test.py`.  For each value of `N`, each process computes the integral of the function for a part of the interval from `a` to `b`.  Namely, the integration for process `rank` of `size` number of processes is between `aProc` and `bProc`, where
    ```python
        N1 = (N* rank)   / size
        N2 =  N*(rank+1) / size
        aProc = a + N1 * h
        bProc = a + N2 * h
    ```
  * Each process uses the `NumPy` function `trapintNumPy` for the calculation of a partial integral value `valProc`, computed between the limits `aProc` and `bProc` and using a number of intervals equal to `N2-N1`.
  * The total value of the integral (between `a` and `b`) is obtained from sum-reducing all the partial results `valProc`.  This is achieved by placing the partial result on each process in a vector of size 1 and by reducing all these partial vectors across all processes to a final result vector on a single process, typically the rank 0.
  * Use the function `MPI.Wtime()` to measure execution time for each value of `N`.
  * The approximated value for the integral (or the absolute error) is printed by the process of rank 0 only, along with the execution time.

    Demonstrate to yourself that the performance of `integrate_mpi.py` scales with the number of processes used (*i.e.* execution time is nearly halved by doubling the number of processes).

6. Finally, run both `integral_test.py` and `integral_mpi.py` once more in order to cross-compare all the 4 solutions.  Thus, for the same number `N` of intervals, make sure that
  * the same level of accuracy in estimating the integral is achieved with all solution;
  * execution time for the `Cython` and `Numba` implementations should be comparable;
  * execution time for the `Cython` and `Numba` implementations (with 2 and 4 threads) and the `mi4py` one (with 2 and 4 processes, respectively) should also be comparable.

## Exercise 3: solution of the 2D heat equation
<hr style="border: solid 4px green">

### The 2D heat equation

#### The maths
The problem is to find the time-varying temperature distribution across a plate given an initial distribution and a fixed temperature around the edges.  Mathematically, this means solving the equation

$$\frac{\partial u}{\partial t}=\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2}$$

subject to
* the initial condition $u(x,y,0)=u_0(x,y)$ and
* the boundary condition $u(x,y,t)=0$ on the boundary.

Assume the domain of the physical problem to be the unit square $0\leq x,y\leq 1$ with the initial conditions

$$u_0(x,y)=\sin\pi x\cdot\sin\pi y$$

With the above, the analytic solution to the mathematical problem is

$$u(x,y,t)=\sin\pi x\cdot\sin\pi y\cdot e^{-2\pi^2 t}$$

which is useful to validate the numerical solution in the Python implementation.

#### The numerical solution
The numerical solution used in this example is via the **F**inite **D**ifference (FD) solution.  Thus, both the space coordinates $0\leq x,y\leq 1$ and time $t\geq 0$ are discretised and the continuous solution $u$ is approximated by a discrete equivalent (an array), which is advanced in time from the initial conditions to a chosen final time value.  In summary, the FD solution means
* sample the 2D domain at equidistant points at coordinates $x_i=i\Delta x$ and $y_j=j\Delta y$
* sample time at points $t_n=n\Delta t$
* compute the discrete values $u^n_{i,j}$ corresponsing to the $x_i$, $y_j$ and $t_n$
* assuming $\Delta x=\Delta y$, the numerical solution is produced by the *time-marching scheme*

$$u^{n+1}_{i,j}=u^{n}_{i,j}+\nu \left( u^{n}_{i+1,j}+u^{n}_{i-1,j}+u^{n}_{i,j+1}+u^{n}_{i,j-1}\ -\ 4u^{n}_{i,j} \right)$$

where $\nu=\frac{\Delta t}{\Delta x^2}\leq 0.25$ for numerical stability.

This numerical scheme results in the 6 point stencil involving *each* discrete point $(i, j)$ at time $t_{n+1}$ is updated from *five* values at time $t_n$ (the same point plus its neighbours).  Diagramatically, this is represented as
<br><br>

<table border="0">
  <tr>
    <td><center>time step $n$</center></td>
    <td><center>time step $n+1$</center></td>
  </tr>
  <tr>
    <td><img src="../images/untxt.png"; style="float: center; width: 40%"></td>
    <td><img src="../images/unp1txt.png"; style="float: center; width: 40%"></td>
  </tr>
</table>

#### The numerical algorithm
The algorithm is
* start with initial conditions
* for each time step
  * for each space point, use current solution $u^{n}$ to compute next step $u^{n+1}$
  * apply boundary conditions
  * swap current solution with updated "next-step" solution

### Setup
The initial Python scripts for this exercise is found in the directory `heat/`, which you are asked to modify using a text editor of your choice.  All Python scripts are to be run directly at the command line.

### Aim
Building on the provided scripts which use only Python, you are also asked to provide alternative implementations of the method scripted using
* `NumPy`
* `Numba`
* `Cython`

### Instructions
Go into the directory `heat/` and follow the following steps:

1. Open the file `heat.py` in a text editor, understand its function and structure.  In particular, understand
   * the function of the two main classes `grid` and `solution`;
   * how the method `setStepper` sets the implementation for the time-stepping method (the stepper);
   * how `timeStep` applies the time-stepping algorithm using the stepper set by `setStepper`.

   A summary of the implementation in `heat.py` is given in the table below
| Class | Data | Methods |
| :--- | :--- | :--- | 
| `grid` | coordinates: `x`, `y` | `error` |
|        | solution: `u`         | |
|        | old solution: `uo`    | |
|  `solution`  | `grid` object | `timeStep` |
|              |               | `setStepper` |
|              |               | `numpyStepper` |
|              |               | `fortranStepper` |
|              |               | `cStepper` |
|              |               | *etc.* |

   The `solution` class has the method `timeStep`, which provides the numerical solution, effectively implementing the update

$$u^{n+1}_{i,j}=u^{n}_{i,j}+\nu \left( u^{n}_{i+1,j}+u^{n}_{i-1,j}+u^{n}_{i,j+1}+u^{n}_{i,j-1}\ -\ 4u^{n}_{i,j} \right)$$

   for all $i$ and $j$, excepting the boundaries.

   `self.stepper` is initialised on one of the methods `*Stepper` available, each one being a different implementation.

   Run the code `heat.py` with the command `python heat.py 100 500` and observe the output.

   *Optional*: using `grid.plot`, try to plot the solution at the end of the iterations.

2. Starting with the class method `pythonStepper`, create a `NumPy` version called `numpyStepper` that implements the same loops as `pythonStepper` but using vectorised operations and vector slicing.  More precisely, `numpyStepper` should implement the operations in the original two loops as a single line of code.

   Also, modify the function `setStepper` so that it can set the `stepper` method to the newly created `numpyStepper` when the variable `stepper` is set to `numpy`.  Then, add the option `numpy` to the `stepperTypeList` list in `main`, so that `main` computes two numerical solutions, one with the pure Pythin implementation, the other with the `NumPy` one.  Run with the same command as before, *i.e.* `python heat.py 100 500`.  What performance gain over a straight `Python` implementations do you see?

3. Write a stepper `numbaStepper`, which is a wrapper for a `Numba` implementation of `pythonStepper`:
   * `numbaStepper` is meant to do nothing else than pass the arguments to a function `numbaStepperJIT` (*N.B.* `numbaStepperJIT` is not a method of the class `solution`);
   * the `Numba` function `numbaStepperJIT` is a direct copy of `pythonStepper` with just the `Numba` `@jit` decorator added.

   Add the newly created stepper `numbaStepper` to the options in `setStepper` and in the list `stepperTypeList`.  Comment out the Python option in `stepperTypeList` (too slow!) and run `heat.py` with the command `python heat.py 1000 500`.  (Recall the implementation has to be run once on a smaller problem size first for the JIT compiler to act and the compiled code to be cached for further use.)  Is the final error comparable with that of the `NumPy` implementations?  What about execution time?

4. Lastly, write a `Cython` implementation of the stepper `cythonStepper`.  Start with a serial `Cython` code and then multi-thread it.

   The new stepper, `cythonStepper`, is a Python wrapper for a separate `Cython` source file called `cython_stepper.pyx`:
   * `cython_stepper.pyx` is a `Cython` version of `pythonStepper`;
   * the module `cython_stepper` is built from `cython_stepper.pyx` using `distutils`;
   * the wrapper `cythonStepper` loads the `cython_stepper` module and passes the arguments to the function in the loaded class from `cython_stepper`.

   In writing the serial `Cython` source pay attention to
   * typing all variables and
   * eliminating the Python tests `boundscheck` and `wraparound`.

   Add the newly created stepper `cythonStepper` to the options in `setStepper` and in the list `stepperTypeList` and test the `Cython` solution.  Run `heat.py` with the command `python heat.py 1000 500`, including only the `NumPy`, `Numba` and `Cython` implementations in the test.  Is the final error of the `Cython` siolution comparable with that of the other implementations?  What about performance?  How does the serial `Cython` solution compare with the `Numba` one?

   Then, add multi-threaded parallelism to the serial `Cython` solution.  To do this
   * load the appropriate classes from the module `cython`;
   * use `cimport` to load `openmp` and provide the underlying parallelism;
   * replace the `range` of the outermost loop (over the `i` index) with `prange`;
   * provide `prange` with the `nogil` option set on `True` and the `num_threads` set on 2.

   Repeat the performance comparison test by re-running `heat.py`.  How does the time for executing the `Cython` solution change?  Is the error comparable?

   Experiment with a higher number of threads (but not higher than the number of cores available on your test system).  Plot a scaling graph, which depicts the variation of the solution time using the `Cython` implementation against the number of threads used in execution.

<img src="../../images/reusematerial.png"; style="float: center; width: 90"; >
<br>
<br>